In [10]:
import pandas as pd
import numpy as np
import nltk
import re

In [11]:
from nltk.corpus import stopwords

In [295]:
dojs=pd.read_csv('jalshakti.txt',header=None,on_bad_lines='skip')
dojs.to_csv('ham1.csv',header=None)

In [296]:
dojs.rename(columns={0:'message'},inplace=True)

In [297]:
dojs['target']=1

In [298]:
dojs

,message,target
0,I'm writing to raise a serious concern about t...,1
1,"Hello, there's an ongoing issue with the water...",1
2,"Greetings, I'd like to report a problem with t...",1
3,"Hi, we've been facing a persistent problem wit...",1
4,I'm concerned about the water quality in our h...,1
...,...,...
1048,I raise this grievance to draw attention to th...,1
1049,Our inability to capture and utilize rainwater...,1
1050,The failure to harness rainwater efficiently h...,1
1051,The loss of rainwater during monsoons is a mat...,1


In [299]:
dort=pd.read_csv('RoadTransport.txt',sep='/t',header=None)

/var/folders/g7/627wzbxx3td29rg7f3r4zdrc0000gn/T/ipykernel_93359/1016892925.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dort=pd.read_csv('RoadTransport.txt',sep='/t',header=None)


In [300]:
dort.rename(columns={0:'message'},inplace=True)

In [301]:
dort['target']=0

In [311]:
dort

,message,target
0,I want to bring to your attention the deterior...,0
1,""" the potholes on our road are causing signifi...",0
2,"""Respected Authorities, I've noticed that the ...",0
3,"""To Whom It May Concern, The potholes near my ...",0
4,"""I am writing to report the dangerous potholes...",0
...,...,...
956,Residents have expressed growing frustration o...,0
957,The roads in our region have fallen into a sta...,0
958,The road conditions are an issue of paramount ...,0
959,I would appreciate your prompt attention to th...,0


In [312]:
list=[dojs,dort]

In [313]:
data=pd.concat(list)

In [314]:
data

,message,target
0,I'm writing to raise a serious concern about t...,1
1,"Hello, there's an ongoing issue with the water...",1
2,"Greetings, I'd like to report a problem with t...",1
3,"Hi, we've been facing a persistent problem wit...",1
4,I'm concerned about the water quality in our h...,1
...,...,...
956,Residents have expressed growing frustration o...,0
957,The roads in our region have fallen into a sta...,0
958,The road conditions are an issue of paramount ...,0
959,I would appreciate your prompt attention to th...,0


In [315]:
data=data.sample(frac=1,random_state=42)

In [316]:
data

,message,target
145,We're lookin' for answers 'bout why these toll...,0
526,Our connection to the river near our residenti...,1
393,"We're concerned about the water supply volume,...",1
354,"The deplorable condition of our local roads, m...",0
433,"Hello, I'm concerned about the water supply is...",1
...,...,...
77,Your attention to this matter is greatly appre...,0
241,It's not just about venting; it's about findin...,0
860,We urgently implore the authorities to expedit...,1
406,I find myself stuck in a dilemma where my dail...,0


In [310]:
data.to_csv("Data.csv")

In [317]:
data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

     

In [318]:
data

,message,target
0,We're lookin' for answers 'bout why these toll...,0
1,Our connection to the river near our residenti...,1
2,"We're concerned about the water supply volume,...",1
3,"The deplorable condition of our local roads, m...",0
4,"Hello, I'm concerned about the water supply is...",1
...,...,...
2009,Your attention to this matter is greatly appre...,0
2010,It's not just about venting; it's about findin...,0
2011,We urgently implore the authorities to expedit...,1
2012,I find myself stuck in a dilemma where my dail...,0


In [319]:
data['message']=data['message'].apply(clean_text)

In [320]:
data['message'][1]

'our connection to the river near our residential area has always been strong  but its currently eperiencing issues like pollution  stagnation  and concerns about our health and environment'

In [321]:
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Bidirectional
from sklearn.model_selection import train_test_split
#from np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from nltk import word_tokenize


In [322]:
from keras.metrics import BinaryAccuracy

In [350]:

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 2500
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 15
# This is fixed.
EMBEDDING_DIM = 5

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['message'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 2576 unique tokens.


In [351]:
data['message'][0]

'were lookin for answers bout why these toll hikes had to happen these new rates  they just dont seem fair at all to folks like us who work hard every day and dont have much etra to spare'

In [352]:

X = tokenizer.texts_to_sequences(data['message'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2014, 15)


In [326]:
X[0]

array([  2,  38, 472, 186, 276, 347, 138, 104,   2,  63,  99, 139, 191,
       375, 496,   3, 347,  39, 473,   2], dtype=int32)

In [353]:
Y=data['target']

In [354]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1812, 15) (1812,)
(202, 15) (202,)


In [355]:

Model = Sequential()
Model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
Model.add(SpatialDropout1D(0.3))
Model.add(LSTM(16, dropout=0.1))
Model.add(Dense(1, activation='sigmoid'))
Model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
print(Model.summary())


Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_35 (Embedding)    (None, 15, 5)             12500     
                                                                 
 spatial_dropout1d_26 (Spat  (None, 15, 5)             0         
 ialDropout1D)                                                   
                                                                 
 lstm_36 (LSTM)              (None, 16)                1408      
                                                                 
 dense_34 (Dense)            (None, 1)                 17        
                                                                 
Total params: 13925 (54.39 KB)
Trainable params: 13925 (54.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [356]:
Model.fit(X_train,Y_train,epochs=5,batch_size=100)

Epoch 1/5
19/19 [==============================] - 1s 4ms/step - loss: 0.6891 - accuracy: 0.5226
Epoch 2/5
19/19 [==============================] - 0s 3ms/step - loss: 0.6596 - accuracy: 0.5541
Epoch 3/5
19/19 [==============================] - 0s 3ms/step - loss: 0.5336 - accuracy: 0.7003
Epoch 4/5
19/19 [==============================] - 0s 3ms/step - loss: 0.3803 - accuracy: 0.9139
Epoch 5/5
19/19 [==============================] - 0s 3ms/step - loss: 0.2909 - accuracy: 0.9415


In [638]:
#Model.evaluate(X_train,Y_train)

57/57 [==============================] - 0s 1ms/step - loss: 0.2418 - binary_accuracy: 0.9961


[0.24177145957946777, 0.9961368441581726]

In [357]:
Model.evaluate(X_test,Y_test)

7/7 [==============================] - 0s 2ms/step - loss: 0.2642 - accuracy: 0.9604


[0.2641524374485016, 0.9603960514068604]

In [333]:
Y_preds=Model.predict(X_test)

7/7 [==============================] - 0s 3ms/step


In [334]:
Y_preds

array([[0.9757262 ],
       [0.07782791],
       [0.10541568],
       [0.9769078 ],
       [0.97359174],
       [0.09725773],
       [0.9624603 ],
       [0.97813416],
       [0.9757306 ],
       [0.97533333],
       [0.9304365 ],
       [0.08367636],
       [0.97409666],
       [0.9767086 ],
       [0.9363654 ],
       [0.08935885],
       [0.08033795],
       [0.97837126],
       [0.89410716],
       [0.56620723],
       [0.7667329 ],
       [0.97146463],
       [0.97358096],
       [0.977435  ],
       [0.0912573 ],
       [0.09933089],
       [0.47988704],
       [0.07759053],
       [0.15616128],
       [0.08005253],
       [0.9607681 ],
       [0.97404987],
       [0.08309243],
       [0.9190914 ],
       [0.971324  ],
       [0.97742647],
       [0.08318725],
       [0.90897906],
       [0.9564762 ],
       [0.07617297],
       [0.07420329],
       [0.30828643],
       [0.0775947 ],
       [0.9768816 ],
       [0.9749554 ],
       [0.9732325 ],
       [0.9678292 ],
       [0.941

In [117]:
for i in range(len(Y_preds)):
    if Y_preds[i]<= 0.5:
        Y_preds[i]=0
    else:
        Y_preds[i]=1

In [118]:
Y_preds

array([[1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [119]:
Y_test

747     1
1880    1
370     1
920     0
1194    1
       ..
1432    0
1342    1
589     0
239     0
1015    0
Name: target, Length: 202, dtype: int64

In [120]:
Y_test.iloc[3]

0

In [121]:
Y_preds[3]

array([0.], dtype=float32)

In [122]:
Y_test.iloc[46]

1

In [123]:
Y_preds[46]

array([1.], dtype=float32)

In [99]:
#the road near my house has alot of potholes, please fix them as it is very risky to drive

In [525]:
user_input=input("enter string")

enter stringWhile I understand the importance of generating revenue for infrastructure maintenance and improvements, I firmly believe that the toll pricing policy should take into consideration the economic realities faced by everyday citizens, especially those in lower income brackets who are disproportionately affected by these elevated toll rates.


In [358]:
def predictor_LSTM():
    string=input(" Enter your grievance: ")
    DF=pd.DataFrame({'message':[string]})
    DF['message']=DF['message'].apply(clean_text)
    
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(X)
    
    x=tokenizer.texts_to_sequences(DF['message'].values)
    x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)
    y=Model.predict(x)
    ans=int(y[0][0]>=0.5)
    L=["Dept of Road,Transport and Highways","Dept of Jal Shakti"]
    
    return L[ans]

In [85]:
predictor_LSTM()

 Enter your grievance: clean ganga
1/1 [==============================] - 0s 14ms/step


'Dept of Jal Shakti'

In [133]:
pickle.dump(Model, open('maglu.pkl', 'wb'))

# ans=int(y[0][0]>=0.5)

In [360]:
def predictor():
    string=input(" Enter your grievance: ")
    DF=pd.DataFrame({'message':[string]})
    DF['message']=DF['message'].apply(clean_text)
    
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(data['message'].values)
    
    x=tokenizer.texts_to_sequences(DF['message'].values)
    x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)
    y=Model.predict(x)
    #ans=int(y[0][0]>=0.6)
    ans=y[0][0]
    
    L=["Dept of Road,Transport and Highways","Dept of Jal Shakti"]
    
    return ans

In [361]:
predictor()

 Enter your grievance: It is with deep concern that I report the deteriorating state of our roads, which is leading to increased frustration and hindrance to daily life and economic progress in our area.
1/1 [==============================] - 0s 153ms/step


0.30237156

In [362]:
predictor()

 Enter your grievance: It is with deep concern that I report the deteriorating state of our roads, which is leading to increased frustration and hindrance to daily life and economic progress in our area.
1/1 [==============================] - 0s 19ms/step


0.30237156

In [363]:
predictor()

 Enter your grievance: clean river ganga project is very slow
1/1 [==============================] - 0s 21ms/step


0.9762607

In [364]:
predictor()

 Enter your grievance: the sanitation is very bad near the handpumps
1/1 [==============================] - 0s 11ms/step


0.9751641

In [365]:
predictor()

 Enter your grievance: i want to raise a concern about the potholes on road near my house
1/1 [==============================] - 0s 11ms/step


0.4202877

In [367]:
predictor()

 Enter your grievance: "I'm concerned about the water quality in our homes. It's been consistently dirty, and we're worried about its impact on our health."
1/1 [==============================] - 0s 64ms/step


0.9799195

In [368]:
predictor()

 Enter your grievance: I wish to express my concern regarding the inadequate street lighting, which poses a threat to our safety during the dark hours. Your intervention is crucial to mitigate these risks and ensure that our neighborhood is safe for all.
1/1 [==============================] - 0s 86ms/step


0.39508915

In [369]:
predictor()

 Enter your grievance: during rain, the canal overflows
1/1 [==============================] - 0s 14ms/step


0.9744891

In [373]:
predictor()

 Enter your grievance: the toll prices are very high
1/1 [==============================] - 0s 24ms/step


0.9735578

In [5]:
import pickle

In [601]:
model_pkl_file = "DEPT_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)

In [ ]:
with open("DEPT_model.pkl",'rb') as file:
    model=pickle.load(file)

In [89]:
process_file="PreProcessing.pkl"

In [ ]:
with open(process_file,'wb') as file:
    pickle.dump(predictor,file)

In [90]:
with open(process_file,'rb') as File:
    func=pickle.load(File)
